In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
import numpy as np
import cv2
import random



tf.logging.set_verbosity(tf.logging.INFO)

# emotion coding: 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise
e_labels = [0, 1, 2, 3, 4, 5, 6, 7]

In [2]:
emotions = []
for path, name, files in os.walk('Emotion'):
    for file in files:
        if(file):
            with open(os.path.join(path, file)) as curf:
                emotions.append((os.path.join(path, file), file[:-4], int(curf.read().strip(' ')[:1])))

print(emotions[0])
img = os.path.join('aligned', emotions[0][1][:-8] + '.png')
print(cv2.imread(img, 0))

print(len(emotions))

('Emotion\\S005\\001\\S005_001_00000011_emotion.txt', 'S005_001_00000011_emotion', 3)
[[  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 ..., 
 [187 189 189 ...,  78  84  92]
 [186 188 186 ...,  87  93  97]
 [192 189 185 ...,  92  98 100]]
327


In [3]:
# let's pick 40 eval samples, too bad this dataset is so small
evals = []
for k in range(40):
    evals.append(random.randrange(326))
    

In [4]:
# want shape (num images, 115,600)
data = []
labels = []
eval_data = []
eval_labels = []


for i in range(len(emotions)):
    im = os.path.join(os.path.join('aligned', emotions[i][1][:-8] + '.png'))
    im_lst = cv2.imread(im, 0)
    if(i in evals):
        eval_data.append(im_lst.flatten())
        eval_labels.append(emotions[i][2])
    else:
        data.append(im_lst.flatten())
        labels.append(emotions[i][2])

npdata = np.array(data, dtype=np.float32)
nplabels = np.array(labels, dtype=np.int32)
npeval_data = np.array(eval_data, dtype=np.float32)
npeval_labels = np.array(eval_labels, dtype=np.int32)

print(npdata.shape)
npdata

(290, 115600)


array([[   0.,    0.,    0., ...,   92.,   98.,  100.],
       [  14.,   13.,   12., ...,    0.,    0.,    0.],
       [  12.,   11.,   11., ...,  145.,  146.,  146.],
       ..., 
       [  41.,   40.,   41., ...,  189.,  190.,  189.],
       [  57.,   59.,   64., ...,   35.,   39.,   45.],
       [  51.,   54.,   56., ...,   39.,   38.,   39.]], dtype=float32)

In [5]:
onehot_labels = tf.one_hot(indices=tf.cast(nplabels[:10], tf.int32), depth=8)
print(onehot_labels.shape)

(10, 8)


In [9]:
# we have 340x340px images
def cnn_model(features, labels, mode):
    input_layer = tf.reshape(features['x'], [-1, 340, 340, 1])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    # shape [batch size, 340, 340, 64]
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    # shape [batch size, 170, 170, 64]
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=128,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    # shape [batch size, 170, 170, 128]
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=1, padding="same")
    # shape [batch size, 81, 81, 64]
    print(tf.shape(pool2))
    
    # padded_pool2 = tf.pad(pool2, [[1, 1], [1, 1]], "CONSTANT")
    # print(tf.shape(padded_pool2))
    pool2_flat = tf.reshape(pool2, [-1, 85 * 85 * 64])
    
    dense=tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(inputs=dense, rate=.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout, units=8)

    predictions = {
        "classes":tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=8)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=.001)
        train_op=optimizer.minimize(
            loss=loss,
            global_stop=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, training_op=train_op)
    
    eval_metric_ops = {
        "accuracy":tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])
    }
    
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [10]:
ck_classifier = tf.estimator.Estimator(model_fn=cnn_model, model_dir="models")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [11]:
tensors_to_log = {"probabilities":"softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":npdata},
    y=nplabels,
    batch_size=10,
    num_epochs=None,
    shuffle=True)

ck_classifier.train(
                input_fn=train_input_fn,
                steps=2000,
                hooks=[logging_hook])

Tensor("Shape:0", shape=(4,), dtype=int32)


ValueError: Shapes (80, 8) and (10, 8) are incompatible

In [ ]:
eval_input_fn=tf.estimator.inputs.numpy_input_fn(
    x={"x":npeval_data},
    y=npeval_labels,
    num_epochs=1,
    shuffle=False)

eval_results=ck_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)